In [6]:
import boto3
import re
import pandas as pd
import numpy as np

input_path = '/home/ec2-user/SageMaker/months-data-raw/'
output_path = '/home/ec2-user/SageMaker/months-parsed-data/'
output_bucket = 'months-parsed-data'

drop =[5,7,11,13,14,18,25,26,27,28,29]
# 1  id integer primary key,
# 2  region_id integer,
# 3  category_id integer,
# 4  subregion_id integer,
# 5  district_id integer,
# 6  city_id integer,
# 7  accurate_location boolean,
# 8  user_id,
# 9  sorting_date datetime,
# 10 created_at_first datetime,
# 11 valid_to datetime,
# 12 title text,
# 13 description text,
# 14 full_description text,
# 15 has_phone boolean,
# 16 params text,
# 17 private_business boolean,
# 18 has_person boolean,
# 19 photo_sizes text,
# 20 paidads_id_index integer,
# 21 paidads_valid_to integer,
# 22 predict_sold datetime,
# 23 predict_replies integer,
# 24 predict_views integer,
# 25 reply_call integer,
# 26 reply_sms integer,
# 27 reply_chat integer,
# 28 reply_call_intent integer,
# 29 reply_chat_intent integer) 
#months = ['2016-12', '2017-01', '2017-02', '2017-03','2017-04', '2017-05']
months = ['2017-01']

col =['id','region_id','category_id', 'subregion_id','city_id',
          'user_id','sorting_date', 'created_at_first',
          'title','has_phone','params','pricate_business',
          'photo_sizes', 'paidads_id_index','paidads_valid_to','predict_sold',
          'predict_replies','predict_views']

failed_entry = []

def replace_f_t(x):
    """
    Replaces strings f and t into binary equivalents 1 and 0.

    :param x: The string to replace.
    :return: The replaced string.
    """
    if x == "f":
        return "0"
    elif x == "t":
        return "1"
    else:
        return x

def drop_columns(entry):
    dropped_count = 1
    for col in drop:
        del entry[col - dropped_count]
        dropped_count += 1
    return entry

def parse_months():
    
    for month in months:
        with open(input_path + month) as infile:
            month_df = np.ndarray(shape=(0,len(col)))
            # Read whole file at once
            file_string = infile.read()

            # Entry columns are separated by ","
            # Entries are separated by "\n"\

            # Sometimes, there will be "","" used in product description, e.g "Movie Title 2"
            # do not split on such cases
            all_columns = re.split('(?<![0-9a-zA-Z?! ]\")\",\"(?!\"[0-9a-zA-Z])|(?<!\n)\"\n\"(?!\")', file_string)

            entry = []
            column_counter = 0
            entry_counter = 0

            for column in all_columns:
                # There are 29 columns in entry
                if column_counter == 29:
                    # Sometimes there are leftover " in first column
                    # get rid of them
                    entry[0] = entry[0].strip('"')

                    # DEBUG - if there are any illegal chars in user product description
                    # it prints where they are so that we can fix/remove them


                    # if is_date(entry[10]) and is_zero_or_one(entry[14]):
                    #    print("OK!")

                    entries = map(replace_f_t, entry)
                    entries = list(entries)
                    entries = [str(el) for el in entries]
                    # Add the entry to the db
                    # WARNING - unfiltered tuple input, assuming that
                    # none of the OLX users is named DROP TABLE ADS
                    entries = drop_columns(entries)
                    if entry_counter != 0:
                        month_df = np.append(month_df, [entries], axis=0)
                        
                    entry = []
                    column_counter = 0

                    entry_counter += 1

                entry.append(column)
                column_counter += 1
            frame = pd.Dataframe(pandas_df)
            frame.to_csv(output_path + month + '.csv',index=False,sep='|')
    return

#len(drop) + len(columns)

In [ ]:
%%time
parse_months()

In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path
from db import db_path
db_path

ModuleNotFoundError: No module named 'db'